In [1]:
import os
import json
import numpy as np
from tqdm.auto import tqdm
import math

In [2]:
from adapted import AdaptedCameras

In [3]:
# constants
REQUESTED_ROOMS = [ 'Bedroom', 'MasterBedroom', 'SecondBedroom']

In [4]:
# folder and path
model_info_json_path = "/home/yizhou/Research/3D_FRONT/3D-FUTURE-model/model_info.json"
scene_folder = "/home/yizhou/Research/3D_FRONT/3D-FRONT/"

In [5]:
# load model category
model_category_info = json.load(open(model_info_json_path, "r"))
model_id2category = {item['model_id']:item['category'] for item in model_category_info}

In [6]:
scene_name_list = os.listdir(scene_folder)

In [7]:
all_room_info = {}
for idx in tqdm(range(len(scene_name_list)//100)):
    scene_file_path = scene_name_list[idx]

    camera = AdaptedCameras(os.path.join(scene_folder, scene_file_path))

    room_dict = camera.scene_content.dict_room

    for iname in room_dict.keys():
        room = room_dict[iname]
        if room.type not in REQUESTED_ROOMS:
            continue
        if iname not in camera.floor_info.keys():
            continue

        
        # get room floor infomation
        floor = camera.floor_info[iname]['floor']
        floor_x = [item[0] for item in floor]
        min_x = np.min(floor_x)
        max_x = np.max(floor_x)
        
        floor_z = [item[1] for item in floor]
        min_z = np.min(floor_z)
        max_z = np.max(floor_z)
        
        # get room bbox
        room_box = [max_x - min_x, 2.5, max_z - min_z] # max: x, y, z
        

        room_dict43DSLN = {'id':iname, 'valid_objects':[], 'bbox':room_box}
        # furniture
        furnitures = room.children_for_furniture
        for ifurniture in furnitures:
            uid = ifurniture['id']
            if uid not in camera.model_dict.keys():
                continue

            model_info = camera.model_dict[uid]
            cate_id = model_info['category_id']

            #if cate_id not in model_id2category:
            #    continue

            cate = model_id2category[model_info['jid']]

            furni_dict = {
                'jid': model_info['jid'],
                'category_id': model_info['category_id'],
                'size': model_info['box'],
                'scale': list(ifurniture['scale']),
                'pos': list(ifurniture['pos']),
                'rot': list(ifurniture['rot']),
                'category':cate,
                'floor': floor
            }
            
            # calculate bbox and normalize
            bbox = [[0,0,0],[0,0,0]]
            bbox[0][0] = furni_dict['pos'][0] - furni_dict['size']['xLen'] / 200.0
            bbox[0][1] = furni_dict['pos'][1]
            bbox[0][2] = furni_dict['pos'][2] - furni_dict['size']['zLen'] / 200.0
            bbox[1][0] = furni_dict['pos'][0] + furni_dict['size']['xLen'] / 200.0
            bbox[1][1] = furni_dict['pos'][1] + furni_dict['size']['yLen'] / 200.0
            bbox[1][2] = furni_dict['pos'][2] + furni_dict['size']['zLen'] / 200.0
            
            furni_dict['unormalized_bbox'] = [[v for v in u] for u in bbox]
            
            bbox[0][0] =  (bbox[0][0] - min_x)/(max_x - min_x)
            #bbox[0][1] =  # bottom y no modification
            bbox[0][2] =  (bbox[0][2] - min_z)/(max_z - min_z)
            bbox[1][0] =  (bbox[1][0] - min_x)/(max_x - min_x)
            bbox[1][1] =  bbox[1][1] / 2.5 # floor height = 2.5
            bbox[1][2] =  (bbox[1][2] - min_z)/(max_z - min_z)
            
            
            furni_dict['new_bbox'] = bbox
            
            
            # get type
            furni_dict['type'] = cate.split(" ")[-1].lower() 
            
            # get rotation
            
            t2 = +2.0 * (furni_dict['rot'][3] * furni_dict['rot'][1] - furni_dict['rot'][2] * furni_dict['rot'][0])
            t2 = +1.0 if t2 > +1.0 else t2
            t2 = -1.0 if t2 < -1.0 else t2
            pitch_y = math.asin(t2) / math.pi * 180
            
            furni_dict['rotation'] = pitch_y // 15 % 24
            
            
            room_dict43DSLN['valid_objects'].append(furni_dict)
        
        if len(room_dict43DSLN['valid_objects']) > 0:
            all_room_info[idx] = room_dict43DSLN
        #break #debug

Warning - process_json_house, furniture 2021159108300402892800/model is repeat
Warning - process_json_house, furniture 2021159108300402892800/model is repeat
Warning - process_json_house, furniture 2021159108300402892800/model is repeat
Warning - process_room, instance 25018/model isn't find
Warning - process_room, instance 25018/model isn't find
Warning - process_room, instance 25018/model isn't find
Warning - process_room, instance 25018/model isn't find
Warning - process_room, instance 25018/model isn't find
Warning - process_room, instance 29662/model isn't find
Warning - process_room, instance 29667/model isn't find
Warning - process_room, instance 29670/model isn't find
Warning - process_room, instance 20317/model isn't find
Warning - process_room, instance 29674/model isn't find
Warning - process_room, instance 29818/model isn't find
Warning - process_room, instance 29828/model isn't find
Warning - process_room, instance 30686/model isn't find
Warning - process_room, instance 23

2021-06-02 14:15:13,608 [ERROR] OtherRoom does not have floor.


Warning - process_room, instance 4775/model isn't find
Warning - process_room, instance 5136/model isn't find
Warning - process_room, instance 5140/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - process_room, instance 4662/model isn't find
Warning - process_room, instance 4665/model isn't find
Warning - process_room, instance 4677/model isn't find
Warning - process_room, instance 4680/model isn't find
Warning - process_room, instance 4684/model isn't find
Warning - process_room, instance 4686/model isn't find
Warning - process_room, instance 4688/model isn't find
Warning - process_room, instance 4689/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - process_room, instance 5248/model isn't find
Warning - 

2021-06-02 14:15:14,268 [ERROR] OtherRoom does not have floor.


Warning - process_room, instance 47507/model isn't find
Warning - process_room, instance 47510/model isn't find
Warning - process_room, instance 47513/model isn't find
Warning - process_room, instance 47513/model isn't find
Warning - process_room, instance 47519/model isn't find
Warning - process_room, instance 47519/model isn't find
Warning - process_room, instance 47539/model isn't find
Warning - process_room, instance 47540/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn't find
Warning - process_room, instance 47542/model isn

...

2021-06-02 14:15:43,688 [ERROR] OtherRoom does not have floor.
2021-06-02 14:15:43,867 [ERROR] OtherRoom does not have floor.


Warning - process_json_house, furniture 2021159401081095388608/model is repeat
Warning - process_json_house, furniture 2021159401081096076480/model is repeat
Warning - process_room, instance 30937/model isn't find
Warning - process_room, instance 30938/model isn't find
Warning - process_room, instance 30939/model isn't find
Warning - process_room, instance 33651/model isn't find
Warning - process_room, instance 191854/model isn't find
Warning - process_room, instance 190531/model isn't find
Warning - process_room, instance 190380/model isn't find
Warning - process_room, instance 213930/model isn't find
Warning - process_room, instance 191854/model isn't find
Warning - process_room, instance 218731/model isn't find
Warning - process_room, instance 191924/model isn't find
Warning - process_room, instance 191924/model isn't find
Warning - process_room, instance 191854/model isn't find
Warning - process_room, instance 190380/model isn't find
Warning - process_room, instance 192097/model is

2021-06-02 14:15:48,262 [ERROR] OtherRoom does not have floor.


Warning - process_room, instance 6351/model isn't find
Warning - process_room, instance 6353/model isn't find
Warning - process_room, instance 8950/model isn't find
Warning - process_room, instance 8953/model isn't find
Warning - process_room, instance 8953/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9550/model isn't find
Warning - process_room, instance 9551/model isn't find
Warning - process_room, instance 7535/model isn't find
Warning - process_room, instance 8045/model isn't find
Warning - process_room, instance 8047/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - process_room, instance 9123/model isn't find
Warning - 

2021-06-02 14:15:48,876 [ERROR] OtherRoom does not have floor.


In [8]:
camera.scene_content

Scene /home/yizhou/Research/3D_FRONT/3D-FRONT/db2b1031-0769-4d04-a2bd-4fe8651c8461.json

In [9]:
all_category_list = []
for oom in all_room_info.values():
    for bject in oom['valid_objects']:
        category = bject['category']
        if category not in all_category_list:
            all_category_list.append(category)

In [10]:
all_category_list = sorted(all_category_list)

In [11]:
all_category_list

['Bed Frame',
 'Bookcase / jewelry Armoire',
 'Ceiling Lamp',
 'Children Cabinet',
 'Corner/Side Table',
 'Desk',
 'Dining Chair',
 'Drawer Chest / Corner cabinet',
 'Dressing Chair',
 'Dressing Table',
 'Footstool / Sofastool / Bed End Stool / Stool',
 'Kids Bed',
 'King-size Bed',
 'Lounge Chair / Cafe Chair / Office Chair',
 'Nightstand',
 'Pendant Lamp',
 'Shelf',
 'Sideboard / Side Cabinet / Console Table',
 'Single bed',
 'TV Stand',
 'Three-Seat / Multi-seat Sofa',
 'Wardrobe',
 'armchair']

In [12]:
all_category_list_cleaned = [item.split(" ")[-1].lower() for item in all_category_list]

In [13]:
valid_types = list(set(all_category_list_cleaned))

In [14]:
json.dump(valid_types, open("metadata43DSLN/valid_types.json", "w"))

In [15]:
json.dump(all_room_info, open("metadata43DSLN/all_room_info.json", "w"))

In [16]:
all_room_info

{0: {'id': 'MasterBedroom-25411',
  'valid_objects': [{'jid': '436914ed-ce9d-48a8-8de2-89c3e4e1d83c',
    'category_id': '41ac92b5-5f88-46d0-a59a-e1ed31739154',
    'size': {'xLen': 184.39999389648438,
     'yLen': 220.15699768066406,
     'zLen': 125.08300018310547},
    'scale': [1.0, 1.0, 1.0],
    'pos': [3.3915216479185357, 0.0, 0.9054263059279903],
    'rot': [0.0, 1.0, 0.0, 0.0],
    'category': 'King-size Bed',
    'floor': [[0.92356, 2.02111],
     [5.46776, 2.02111],
     [5.46776, -1.36196],
     [0.92356, -1.36196],
     [0.92356, 2.02111]],
    'unormalized_bbox': [[2.469521678436114, 0.0, 0.28001130501246296],
     [4.313521617400958, 1.1007849884033203, 1.5308413068435178]],
    'new_bbox': [[0.3402054659645512, 0.0, 0.48534949173752334],
     [0.745997451124721, 0.4403139953613281, 0.8550817177426179]],
    'type': 'bed',
    'rotation': 0.0},
   {'jid': '57f7e7c0-8ca3-4eee-b127-5a44c76a9a61',
    'category_id': '2dd2368f-a3eb-43c2-8a6b-d585cd19d9a6',
    'size': {'xLen